In [3]:
# default_exp algo.dl.loss

%reload_ext autoreload
%autoreload 2

# algo-dl-loss

## 基类 `torch.nn.modules.loss._Loss`

loss也继承了Module

注意`__init__`中的size_average, reduce已经被废弃不再使用

* reduction :计算模式，可为none/sum/mean  


    none- 逐个元素计算
    sum- 所有元素求和，返回标量
    mean- 加权平均，返回标量

In [ ]:
class _Loss(Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction

## 分类loss

### nn.CrossEntropyLoss
功能: nn.LogSoftmax()与nn.NLLLoss()结合，进行交叉熵计算

主要参数:
• weight:各类别的loss设置权值

• ignore_index:忽略某个类别

• reduction :计算模式，可为none/sum/mean 

none- 逐个样本计算
sum- 所有样本求和，返回标量
mean- 加权平均，返回标量


对于固定的某个样本i的预测值输出x,x为输出向量，对应位置的元素代表对应类别的值，class代表真实的类别，

$$H(P,Q)=-\sum_{i=1}^NP(x_i)logQ(x_i)\\=-logQ(x_i)\\=-log(softmax(x))\\=-log(\frac{exp(x[class])}{\sum_jexp(x[j])})
\\=-x[class]+log(\sum_jexp(x[j]))$$




LogSoftmax： 先进行softmax然后再取log


#### 熵(信息熵)|相对熵|交叉熵
自信息: I(x)。用来衡量单个事件的不确定性。
$$I(x)=-log P(x)$$
P(x)为事件x发生的概率。由公式的，如果x是确定事件P(x)=1，则I(x)=0，P(x)越小，I(x)越大，最大会到正无穷

信息熵H(P): 自信息的期望。用来描述信息的不确定性。信息越不确定，熵越大
$$E_{x~p}[I(x)]=-\sum_i^NP(x_i)log P(x_i)$$
即独立同分布的多个事件x1...xN, 这些事件的自信息期望


相对熵(KL散度): 用来衡量两个分布之间的差异(距离)。其中P为真实分布，Q为预测值分布。但是它不是距离函数，因为不具备对称性。
$$D_{KL}(P,Q)=E_{x~p}[log\frac{P(x)}{Q(x)}]\\=E_{x~p}[logP(x)-logQ(x)]\\=\sum_{i=1}^NP(x_i)[logP(x_i)-logQ(x_i)]\\=\sum_{i=1}^NP(x_i)logP(x_i)-\sum_{i=1}^NP(x_i)logQ(x_i)\\=H(P,Q)-H(P)$$

交叉熵 = 信息熵+相对熵
$$H(P,Q)=D_{KL}(P,Q)+H(P)=-\sum_{i=1}^NP(x_i)logQ(x_i)$$
由于P是给定的真实值的分布，所以H(P)为定值，优化交叉熵即等价于优化相对熵

对于一个样本xi,因为它已经发生了，所以P(xi)=1, 模型预测其发生的概率为y_pred, 那么交叉熵为-log(y_pred)

#### 代码实现

In [6]:
import torch

In [59]:
loss = torch.nn.CrossEntropyLoss()

$$\text{loss}(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)
                   = -x[class] + \log\left(\sum_j \exp(x[j])\right)$$

In [26]:
y_pred = torch.from_numpy(np.random.rand(2, 3))  # batch_size, num_classes

In [27]:
y_pred

tensor([[0.3811, 0.8779, 0.5579],
        [0.6266, 0.7077, 0.7452]], dtype=torch.float64)

In [35]:
target = torch.from_numpy(np.array([1, 2]))

In [60]:
loss(y_pred, target)

tensor(1.3107, dtype=torch.float64)

In [39]:
loss = torch.nn.CrossEntropyLoss(reduction='none')

In [40]:
loss(y_pred, target)

tensor([0.8479, 1.0478], dtype=torch.float64)

In [43]:
# 计算第0个样本的交叉熵损失
-y_pred[0, 1] + torch.log(torch.exp(y_pred[0, 0])+torch.exp(y_pred[0, 1])+torch.exp(y_pred[0, 2]))

tensor(0.8479, dtype=torch.float64)

## nn.NLLLoss 
功能:实现负对数似然函数中的负号功能


In [46]:
loss = torch.nn.NLLLoss()

In [47]:
y_pred = torch.from_numpy(np.random.rand(2, 3))  # batch_size, num_classes

In [48]:
y_pred

tensor([[0.9556, 0.2573, 0.8668],
        [0.6220, 0.2094, 0.5222]], dtype=torch.float64)

In [50]:
m = torch.nn.LogSoftmax(dim=1)

In [51]:
m(y_pred)

tensor([[-0.8806, -1.5790, -0.9695],
        [-0.9427, -1.3553, -1.0425]], dtype=torch.float64)

In [55]:
target

tensor([1, 2])

In [57]:
loss(m(y_pred), target)

tensor(1.3107, dtype=torch.float64)

In [52]:
aa = torch.nn.Softmax(dim=1)

In [53]:
aa(y_pred)

tensor([[0.4145, 0.2062, 0.3793],
        [0.3896, 0.2579, 0.3526]], dtype=torch.float64)

## nn.BCELoss 
功能:二分类交叉熵

注意事项:输入值取值在[0,1]

In [61]:
loss = torch.nn.BCELoss()

In [62]:
m = nn.Sigmoid()
y_pred = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(y_pred), target)
output.backward()

In [63]:
y_pred

tensor([-0.0447,  1.6406, -1.1389], requires_grad=True)

## nn.BCEWithLogitsLoss 
功能:结合Sigmoid与二分类交叉熵

注意事项:网络最后不加sigmoid函数

nn.L1Loss
功能: 计算inputs与target之差的绝对值
6、nn.MSELoss
功能: 计算inputs与target之差的平方
7、 SmoothL1Loss
8、PoissonNLLLoss 
功能:泊松分布的负对数似然损失函数

nn.KLDivLoss 
功能:计算KLD(divergence)，KL散度，相对熵
注意事项:需提前将输入计算 log-probabilities， 如通过nn.logsoftmax()
主要参数:

10、nn.MarginRankingLoss 功能:计算两个向量之间的相似度，用于排序任务
特别说明:该方法计算两组数据之间的差异，返回一个n*n 的 loss 矩阵

11、nn.MultiLabelMarginLoss

12、 nn.SoftMarginLoss 功能:计算二分类的logistic损失

13、nn.MultiLabelSoftMarginLoss 功能:SoftMarginLoss多标签版本

14、nn.MultiMarginLoss 功能:计算多分类的折页损失

15、nn.TripletMarginLoss 功能:计算三元组损失，人脸验证中常用

16、nn.HingeEmbeddingLoss 功能:计算两个输入的相似性，常用于
非线性embedding和半监督学习
特别注意:输入x应为两个输入之差的绝对值

17、nn.CosineEmbeddingLoss 功能:采用余弦相似度计算两个输入的相似性

18、nn.CTCLoss
功能: 计算CTC损失，解决时序类数据的分类
Connectionist Temporal Classification

## Focal Loss
https://zhuanlan.zhihu.com/p/32423092 苏剑林

https://zhuanlan.zhihu.com/p/49981234

本质上讲，Focal Loss 就是一个解决分类问题中类别不平衡、分类难度差异的一个 loss

通过减少易分类样本的权重，使得模型在训练时更专注于难分类的样本。

$$L_{focalloss}=-\alpha(1-\hat y_t)^\gamma log\hat y_t$$

### 传统的CrossEntropy的缺点
$$L_{CE}=-log\hat y_t$$

* 首先 对于样本不均衡情况，希望类别较少的样本在计算loss时权重大一点，这在GBDT中已经考虑了，并不稀奇。

* 还有就是希望分错的样本或者区分度差的样本，在计算loss时权重大一点。最简单的方式就是乘以(1-\hat y)就可以达到效果，这里\hat y表示取得对应分类的概率，
    *当\hat y趋近于1时，表示模型已经很容易区分这个样本了，希望它的权重小一些
    *而\hat y离1远时表示模型对该样本的区分度不好，希望它的权重大一些
    
至于(1-\hat y)^\gamma，系数\gamma就是用来调节权重的比例，\gamma越大，模型越倾向于区分度不好的样本。
* 当\gamma=1时，\hat y=0.9, \hat y=0.1， 二者的权重相差9倍，  
* 当\gamma=2时，\hat y=0.9, \hat y=0.1， 二者的权重相差81倍，  

In [125]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1708.02002', width=1200, height=550)

### 实现
https://ptorch.com/news/253.html

In [ ]:
def MyFocalLoss(preds, labels, class_weights=[1, 2], gamma=1):
    """
    preds = torch.tensor([[-0.0450, -0.0435],
        [ 0.0063,  0.0288],
        [ 0.2572, -0.1292],
        [ 0.1535, -0.1328],
        [ 0.2729, -0.2122],
        [ 0.1859, -0.1356],
        [ 0.1851,  0.0081],])
    labels = torch.tensor([0, 0, 1, 1, 0, 1, 0,])
    """
    class_weights = torch.tensor(class_weights).to('cuda')
    class_weights = class_weights.gather(0, labels)
    preds = F.softmax(preds, -1)
    t = torch.gather(preds, 1, labels.unsqueeze(1))
    loss = -(1-t)**gamma * torch.log(t)
    loss = loss.squeeze() * class_weights
    return loss.mean()


In [ ]:
# -*- coding: utf-8 -*-
# @Author  : LG
from torch import nn
import torch
from torch.nn import functional as F

class focal_loss(nn.Module):    
    def __init__(self, alpha=0.25, gamma=2, num_classes = 3, size_average=True):
        """
        focal_loss损失函数, -α(1-yi)**γ *ce_loss(xi,yi)      
        步骤详细的实现了 focal_loss损失函数.
        :param alpha:   阿尔法α,类别权重.      当α是列表时,为各类别权重,当α为常数时,类别权重为[α, 1-α, 1-α, ....],常用于 目标检测算法中抑制背景类 , retainnet中设置为0.25
        :param gamma:   伽马γ,难易样本调节参数. retainnet中设置为2
        :param num_classes:     类别数量
        :param size_average:    损失计算方式,默认取均值
        """

        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   # α可以以list方式输入,size:[num_classes] 用于对不同类别精细地赋予权重
            print("Focal_loss alpha = {}, 将对每一类权重进行精细化赋值".format(alpha))
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1   #如果α为一个常数,则降低第一类的影响,在目标检测中为第一类
            print(" --- Focal_loss alpha = {} ,将对背景类进行衰减,请在目标检测任务中使用 --- ".format(alpha))
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha) # α 最终为 [ α, 1-α, 1-α, 1-α, 1-α, ...] size:[num_classes]
        self.gamma = gamma

    def forward(self, preds, labels):
        """
        focal_loss损失计算        
        :param preds:   预测类别. size:[B,N,C] or [B,C]    分别对应与检测与分类任务, B 批次, N检测框数, C类别数        
        :param labels:  实际类别. size:[B,N] or [B]        
        :return:
        """        
        # assert preds.dim()==2 and labels.dim()==1        
        preds = preds.view(-1,preds.size(-1))        
        self.alpha = self.alpha.to(preds.device)        
        preds_softmax = F.softmax(preds, dim=1) # 这里并没有直接使用log_softmax, 因为后面会用到softmax的结果(当然你也可以使用log_softmax,然后进行exp操作)        
        preds_logsoft = torch.log(preds_softmax)
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   # 这部分实现nll_loss ( crossempty = log_softmax + nll )        
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))        
        self.alpha = self.alpha.gather(0,labels.view(-1))        
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ
        loss = torch.mul(self.alpha, loss.t())        
        if self.size_average:        
            loss = loss.mean()        
        else:            
            loss = loss.sum()        
        return loss

## contrastive loss对比损失
http://www.leonwang.top/2019/07/21/Siamese-Network/

主要用于孪生网络中

$$L=\frac{1}{2 N} \sum_{n=1}^{N} y d^{2}+(1-y) m a x(\text {margin}-d, 0)^{2}$$

其中d=|an−bn|2，代表两个样本特征的欧氏距离，y为两个样本是否匹配的标签，y=1代表两个样本相似或者匹配，y=0则代表不匹配，margin为设定的阈值。

这种损失函数最初来源于Yann LeCun的Dimensionality Reduction by Learning an Invariant Mapping，主要是用在降维中，即本来相似的样本，在经过降维（特征提取）后，在特征空间中，两个样本仍旧相似；而原本不相似的样本，在经过降维后，在特征空间中，两个样本仍旧不相似。

观察上述的contrastive loss的表达式可以发现，这种损失函数可以很好的表达成对样本的匹配程度，也能够很好用于训练提取特征的模型。当y=1（即样本相似）时，损失函数只剩下∑yd2，即原本相似的样本，如果在特征空间的欧式距离较大，则说明当前的模型不好，因此加大损失。而当y=0时（即样本不相似）时，损失函数为∑(1−y)max(margin−d,0)2，即当样本不相似时，其特征空间的欧式距离反而小的话，损失值会变大，这也正好符合我们的要求。

In [ ]:
from nbdev.export import notebook2script
notebook2script('.ipynb')